In [1]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Scrapers.PERIScraper import PeriScraper
from PyClasses.Preprocessing.PRPMCleaner import PRPMCleaner

PRPMscrap = PRPMScraper()
prpm_cleaner = PRPMCleaner()

word = 'yang'
result = PRPMscrap.findWordMetaData(word)
print(result)

if result and 'meanings' in result[word]:
    raw_meanings = result[word]['meanings']
    cleaned_meanings = prpm_cleaner.clean_all_meanings(raw_meanings)
    result[word]['meanings'] = cleaned_meanings

# Output cleaned result
print("Cleaned Result:", result)

#found smth can be preprocess if not done here, like there will have '\xa' character in the result string

/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


{'yang': {'meanings': ['(Lin) kata penghubung utk menyatakan bahawa kata keterangan berikutnya dipentingkan atau menunjukkan kelainan', '(Lin) kata penghubung utk menyatakan bahawa katakata (ayat) berikutnya ialah penjelasan utk kata-kata sebelumnya', 'kata yg digunakan sbg kata penyerta, kata yg menentukan kata benda, kata sandang', 'sl adapun akan', 'bp kata peng-hubung utk menyatakan isi atau huraiankata (ayat) sebelumnya, bahawa', '= yang-yang dewa'], 'synonym': []}}
Cleaned Result: {'yang': {'meanings': ['(lin) kata penghubung untuk menyatakan bahawa kata keterangan berikutnya dipentingkan atau menunjukkan kelainan', '(lin) kata penghubung untuk menyatakan bahawa katakan (ayat) berikutnya ialah penjelasan untuk kata-kata sebelumnya', 'kata yang digunakan sebagai kata penyerta, kata yang menentukan kata benda, kata sandang', 'sama adapun akan', 'bapa kata penpergi-hubung untuk menyatakan isi atau huraiankata (ayat) sebelumnya, bahawa', '= yang-yang dewa'], 'synonym': []}}


In [3]:
from datetime import datetime, timedelta
from PyClasses.Scrapers.BHarianScraper import BHarianScraper
from pyspark.sql import SparkSession
from selenium import webdriver

def ArticleDataCollection(fromDate,categoryLink):
    spark = SparkSession\
        .builder\
        .appName("BHarian WebScraping")\
        .getOrCreate()

    ## crawl links
    bharianScraper = BHarianScraper()
    timeNow = datetime.now()
    crawlledUrls = bharianScraper.scrapArticleLinks(fromDate,categoryLink)
    url_RDD = spark.sparkContext.parallelize(crawlledUrls)
    if crawlledUrls is None:
        print('failed to get article links')
        return 'fail'
    
    ## parallelize process to scrap data
    scrappedData = url_RDD.map(BHarianScraper.scrapOneArticle).collect()
    for data in scrappedData:
        if data is None:
            scrappedData.remove(data)
            
    ## Save Data into HDFS
    df = spark.createDataFrame(scrappedData)
    df.write.format("parquet").mode("overwrite").save("DE-prj/RawData")
    timeEnd = datetime.now() 
    timeTaken = timeEnd - timeNow
    print(f'Processing Start Time: {timeNow}')
    print(f'Processing End Time: {timeEnd}')
    print(f'Total Time Taken: {timeTaken}')
    
    print('done')
    return 'success'

ArticleDataCollection(datetime(2024,12,10),'/berita/nasional')

24/12/14 02:10:15 WARN Utils: Your hostname, YN. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/14 02:10:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/14 02:10:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Article teaser found on page 0.
Current no. of link: 20
Article teaser found on page 1.
Current no. of link: 39
Article teaser found on page 2.
Current no. of link: 58
Article teaser found on page 3.
Current no. of link: 77
Article teaser found on page 4.
Current no. of link: 96
Article teaser found on page 5.
Current no. of link: 115
Article teaser found on page 6.
Current no. of link: 134
Article teaser found on page 7.
Current no. of link: 153
Article teaser found on page 8.
Current no. of link: 172
Article teaser found on page 9.
Current no. of link: 191
Article teaser found on page 10.
Current no. of link: 210
Article teaser found on page 11.
Current no. of link: 229
Article teaser found on page 12.
Current no. of link: 248
Article teaser found on page 13.
Current no. of link: 267
Article teaser found on page 14.
Current no. of link: 286
Article teaser found on page 15.
Current no. of link: 305
Article teaser found on page 16.
Current no. of link: 324
Article teaser found on page 

'author_n_dateTimeEle' element on '/berita/nasional/2024/12/1336692/banjir-amanah-tangguh-konvensyen-nasional-2024' not found. Scrap Attempt:1 time(s)
Timeout error on '/berita/nasional/2024/12/1336148/banjir-dungun-pulih-hanya-kemaman-terkesan-dengan-4-pps-masih-dibuka'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1335119/th-sumbang-rm500000-bantu-mangsa-banjir'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1335376/cadangan-penyelarasan-harga-baharu-beras-putih-tempatan-belum'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1335688/batu-puteh-kertas-kabinet-dibentangkan-tak-ada-orang-tanya-soalan'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1336028/aplikasi-meta-alami-gangguan-global'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1336631/notis-kebankrapan-saya-sudah-beri-peluang-muhyiddin-bayar-rm1402'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/

Processing Start Time: 2024-12-14 02:10:28.617531
Processing End Time: 2024-12-14 02:24:22.880305
Total Time Taken: 0:13:54.262774
done


'success'

In [4]:
# for link in bharianScraper.relLinkToArticle_all:
#     print(link)
spark = SparkSession\
        .builder\
        .appName("ReadRawData")\
        .getOrCreate()
df_read = spark.read.parquet("DE-prj/RawData")
df_read.show()


df_filter = df_read.where(df_read['title'] == '')
df_filter.show()

print(df_read.count())


# Get the first 5 rows
rows = df_read.take(125)

# Print each row
print(rows[15])

24/12/14 02:35:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|       authorDetails|         publishTime|        articleBrief|            contents|                 url|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Kalau anda sayang...|{authName -> Nor ...|Disember 10, 2024...|Ketua Pergerakan ...|[PASIR MAS: UMNO ...|/berita/nasional/...|
|Dungun daerah ked...|{authName -> Nazd...|Disember 10, 2024...| Penduduk membawa...|[KUALA TERENGGANU...|/berita/nasional/...|
|Gelombang kedua: ...|{authName -> BERN...|Disember 10, 2024...|Keadaan rumah mas...|[KUALA LUMPUR: Pa...|/berita/nasional/...|
|Diskriminasi caj ...|{authName -> Fahm...|Disember 10, 2024...|Ahli Parlimen Pul...|[KUALA LUMPUR: Se...|/berita/nasional/...|
|Sanusi lega bebas...|{authName -> Dawn...|Disember 10, 2024...|Menteri Besar Ked...|[SHAH ALAM: Mente..

In [5]:
for data in rows:
    print(data)

Row(title='Kalau anda sayangkan UMNO, kembalilah semula kepada UMNO - Dr Akmal', authorDetails={'authName': 'Nor Fazlina Abdul Rahim', 'relLink': '/authors/nor-fazlina-abdul-rahim', 'email': 'nfazlina@nstp.com.my'}, publishTime='Disember 10, 2024 @ 1:16pm', articleBrief='Ketua Pergerakan Pemuda UMNO, Datuk Dr Muhamad Akmal Saleh pada sidang media selepas melakukan Misi Bantuan Banjir di Kampung Serongga, Pasir Mas, Kelantan hari ini. - Foto NSTP/Nik Abdullah Nik Omar ', contents=['PASIR MAS: UMNO mengalu-alukan bekas pimpinan parti itu termasuk Datuk Seri Hishammuddin Hussein, Khairy Jamaluddin dan Tan Sri Noh Omar kembali menyertai parti itu sekiranya masih sayang kepada parti. ', 'Ketua Pergerakan Pemuda UMNO, Datuk Dr Muhamad Akmal Saleh, berkata UMNO dengan hati terbuka menerima balik mana-mana pimpinan terutama mereka yang pernah berjasa. ', '"Saya mengalu-alukan sesiapa sahaja termasuk Datuk Seri Hishammuddin Hussein, Khairy Jamaluddin, Tan Sri Noh Omar dan Shahril Hamdan sekiran

# PERIBAHASA STARTS HERE YA !

In [ ]:
peri_scrap = PeriScraper()

proverbs_data_AM = peri_scrap.scrape_page(peri_scrap.AM_Url)
print("Proverbs from A–M:")

for proverb, meaning in proverbs_data_AM:
    print(f"Peribahasa: {proverb} || Maksud: {meaning}")

proverbs_data_NZ = peri_scrap.scrape_page(peri_scrap.NZ_Url)
print("\nProverbs from N–Z:")

for proverb, meaning in proverbs_data_NZ:
    print(f"Peribahasa: {proverb} || Maksud: {meaning}")